# You Have to run DataDownload.ipynb First!

In [1]:
import pandas as pd
import h5py
import numpy as np
import nibabel as nib
from tqdm import tqdm
from pathlib import Path
import os
import shutil
import os
import zipfile

In [2]:
csv_path = '/root/Data/BraTS2020_training_data/content/data/meta_data.csv'
target_path = '/root/Data/BraTS2020_training_data//BraTS2020_converted/'

folders_to_create = ['image', 'label']

# 각 폴더 생성
for folder in folders_to_create:
    folder_path = os.path.join(target_path, folder)
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f'Created folder: {folder_path}')
    else:
        print(f'Folder already exists: {folder_path}')

# CSV 파일 로드
df = pd.read_csv(csv_path)

Created folder: /root/Data/BraTS2020_training_data//BraTS2020_converted/image
Created folder: /root/Data/BraTS2020_training_data//BraTS2020_converted/label


In [3]:
# volume 별로 그룹화
grouped = df.groupby('volume')

# 각 volume 그룹을 순회하며 HDF5 파일을 결합하고 저장
for volume, group in tqdm(grouped, desc="Processing volumes"):
    if os.path.exists(f'{target_path}image/volume_{volume}_image.nii'):
        continue
    
    image_slices = []
    mask_slices = []

    for _, row in group.iterrows():
        with h5py.File("/root/Data/BraTS2020_training_data/"+ row['slice_path'], 'r') as file:
            image_slices.append(file['image'][:])
            mask_slices.append(file['mask'][:])

    # numpy 배열로 변환
    image_data = np.stack(image_slices, axis=-1)
    mask_data = np.stack(mask_slices, axis=-1)

    # .nii 파일로 저장
    image_nii = nib.Nifti1Image(image_data, np.eye(4))
    mask_nii = nib.Nifti1Image(mask_data, np.eye(4))

    nib.save(image_nii, f'{target_path}image/volume_{volume}_image.nii')
    nib.save(mask_nii, f'{target_path}label/volume_{volume}_mask.nii')

Processing volumes:   0%|          | 0/369 [00:00<?, ?it/s]

Processing volumes:  18%|█▊        | 67/369 [10:43<45:27,  9.03s/it]  

In [ ]:
def compress_directory(src, dest_zip):
    # Compress the source directory into a zip file
    shutil.make_archive(dest_zip, 'zip', src)

In [ ]:
def copy_file(src, dest):
    # Copy the zip file to the destination directory
    shutil.copy(src, dest)

In [ ]:
compressed_file = '/root/Data/BraTS2020_training_data/compressed_file'  # Temporary location for compressed file
compressed_file_zip = compressed_file + '.zip'
drive_path = '/content/drive/MyDrive/DeepLearningData/'

In [ ]:
# Compress the source directory
compress_directory(target_path, compressed_file)

In [ ]:
copy_file(compress_directory, drive_path)